In [ ]:
# 학습된 모델 확인

import time, json
from typing import Literal, Callable
import random
from random import randint

from drivingppo.common import set_seed, DIS_SCFAC, WORLD_DT, ACTION_REPEAT, LOOKAHEAD_POINTS
from drivingppo.world import World, Car, OBSTACLE_VALUE, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.environment import WorldEnv
from drivingppo.model import run

import numpy as np

from stable_baselines3 import PPO

LOG_DIR = "./logs/"
CHECKPOINT_DIR = './checks/'

from world_samples import NEAR, W_CONFIG
from world_samples import gen_from, gen_1t, gen_2t, gen_3t, gen_3l, gen_obs
from world_samples import (
    generate_random_world_plain,
    generate_random_world_obs_matrix,
    generate_random_world_narrow,
    generate_random_world_obs_between,
    generate_world_simpleLine,
    generate_world_square,
    generate_world_zipper,
)

import math
pi = math.pi
pi2 = pi*2


MAP_W, MAP_H = 150, 150

def tw():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 40),
            (20, 40),
            (50, 60),
            (60, 50),

            # (10, 50),
            # (30, 70),
            # (70, 50),
            # (90, 70),

            # (10, 30),
            # (10, 50),
            # (10, 70),
            # (40, 70),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w

def gen_3l(): return generate_random_world_plain(map_h=300, map_w=300, num=10,               min_dist=NEAR*2, max_dist=DIS_SCFAC, ang_lim=pi*1.0, ang_init='p',    spd_init=0.0)  # 테스트용: 같은 패턴인데 좀 긴 경로이고 초기 0

name = 'wwwwwwwwwwwwwwwwwwwwwwwwww'

# set_seed(10)
# run(gen_3t, name, auto_close_at_end=False)
# run(gen_3l, name, auto_close_at_end=False)
run(tw, name, auto_close_at_end=False)
# run(gen_from(gen_3l, 0, 1), name, auto_close_at_end=False)
# run(tw, f'wwwwwwwwwwwwwwwwwwwwwwwwww', auto_close_at_end=False)

print('#########')

In [ ]:
a = 0.95
a**50

In [ ]:
a = [
]
a.sort()
a

In [ ]:
# 각 모델 평가(모든 에피소드 성적)
from drivingppo.model import evaluate
from drivingppo.common import set_seed
# from world_samples import generate_random_world_plain
from world_samples import gen_3l, generate_random_world_plain, NEAR, DIS_SCFAC
import math; pi = math.pi

# def gen_3l(): return generate_random_world_plain(map_h=300, map_w=300, num=10,               min_dist=NEAR*2, max_dist=DIS_SCFAC, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')  # 테스트용: 같은 패턴인데 좀 긴 경로

# name = 'wwwwwwwwwwwwwwwwwwwwwwwwww'
# set_seed(0)
# a = evaluate(name, gen_3l, 100,
#              csv_path='results/'+name+'.csv'
#              )

def a(name0:str):
    for seed in range(100, 501, 100):
        name = f'{name0}-{seed}'
        print(f'\n\nNAME: {name}')

        set_seed(0)
        evaluate(name, gen_3l, 100,
                    csv_path='results/'+name+'.csv'
                    )

a('m95-fagnt-shwp08')

In [ ]:
# 평가 모으기(모델별 성적)
import os, glob
from pathlib import Path

import pandas as pd

# TODO configId 맞춰
def aggregate_csv_data__old(directory_path):
    # 디렉토리 내 모든 CSV 파일 경로 탐색
    search_pattern = os.path.join(directory_path, '**', '*.csv')
    csv_files = glob.glob(search_pattern, recursive=True)
    
    if not csv_files:
        raise ValueError(f"'{directory_path}' 경로에 CSV 파일이 존재하지 않습니다.")

    processed_rows = []

    for file_path in csv_files:
        id = str(Path(os.path.basename(file_path)).with_suffix(''))
        idParts = id.split('-')
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"!!!!!!!! 읽기 실패: {file_path} ... {e}")
            continue

        N     = 4  if idParts[0][0] == 'y'  else 20
        G     = idParts[0][1:]  if len(idParts[0][1:]) > 0  else 100
        Coord = 'ACC'  if idParts[1] == 'fagnt'  else 'SRC'
        FE    = 'I'  if idParts[2] == 'nofe'  else 'WSWE'  if idParts[2] == 'shwp'  else 'VMLP'  if idParts[2] == 'vmlp'  else idParts[2]
        seed  = idParts[3]
        S = ''
        configId = f'N{N}-G{G}-{Coord}-{FE}{S}'
        row_data:dict[str, float|str] = {
            'id':    id,
            'configId': configId,
            'N':     20,
            'gamma': G,
            'Coord': Coord,
            'FE':    FE,
            'seed':  seed,
        }

        # 수 컬럼: 평균 계산
        numeric_cols = df.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            row_data[col] = df[col].mean()

        # 텍스트 컬럼: 값별 비율 계산
        categorical_cols = df.select_dtypes(exclude=['number']).columns
        for col in categorical_cols:
            proportions = df[col].value_counts(normalize=True, dropna=False)
            for category_val, proportion in proportions.items():
                column_name = f"{col}/{category_val}"
                row_data[column_name] = proportion

        processed_rows.append(row_data)

    df = pd.DataFrame(processed_rows)
    df = df.fillna(0)  # 특정 파일에서만 있는 범주가 다른 파일에서는 NaN 처리되므로 0으로 채움.

    return df

df = aggregate_csv_data__old('./results')
df.to_csv('./results.csv', index=False)

In [ ]:
# 성적 상자수염
from typing import Any

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

df = pd.read_csv('./results.csv')
# COLS = ['id', 'configId', 'N', 'gamma', 'Coord', 'FE', 'seed','ending/sec','ending/sec','metrics/normed_path_len','metrics/brake_rate']
mask = df['FE'] == 'I'
df = df[mask]

grouped = df.groupby('configId')
labels = list(grouped.groups.keys())

def addBoxplot(row, col, title:str, data:list[pd.Series], vl=0):
    median_style = {'color': 'black', 'linewidth': 1.5}
    bplot = axes[row][col].boxplot(data, tick_labels=labels, whis=(0, 100), patch_artist=True, medianprops=median_style)
    for patch, label in zip(bplot['boxes'], labels):
        patch.set_facecolor('#d50') if 'ACC' in str(label) else \
        patch.set_facecolor('#4df')
    axes[row][col].set_title(title, fontsize=18)
    axes[row][col].tick_params(axis='x', rotation=45)
    axes[row][col].grid(True, axis='y', linestyle='--')

    if vl > 0:
        for i in range(1, len(data) // vl +1):
            x = i * vl + 0.5
            axes[row][col].axvline(x=x, color='gray', linestyle=':', alpha=0.7)

fig, axes = plt.subplots(2, 2, figsize=(12, 13))
addBoxplot(0, 0, 'Episode Duration (s)',          [grouped.get_group(label)['ending/sec'             ].dropna() for label in labels], 2)
addBoxplot(0, 1, 'Normalized Path Length',        [grouped.get_group(label)['metrics/normed_path_len'].dropna() for label in labels], 2)
addBoxplot(1, 0, 'Longitudinal Action Variation', [grouped.get_group(label)['metrics/ws_diff_mean'   ].dropna() for label in labels], 2)
addBoxplot(1, 1, 'Speed Variance',                [grouped.get_group(label)['metrics/speed_var'      ].dropna() for label in labels], 2)

legend_elements = [
    Patch(facecolor='#e50', edgecolor='black', label='ACC'),
    Patch(facecolor='#4df', edgecolor='black', label='SRC')
]
fig.legend(handles=legend_elements, loc='upper center', ncol=2, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.savefig('result.png')
plt.show()
